In [1]:
# Loading and Saving data

In [3]:
# TEXT
inputRDD = sc.textFile('hdfs:///user/pgoyal/spark_files/Readme.md')

sc.wholeTextFiles() : to save multiple files from a directory. returns a pair RDD with key as name of input file
# used in cases of sales data time-wise

In [8]:
# JSON 

# Load as text file, and then map over the values with the JSON parser
import json
inputJson = sc.textFile('hdfs:///user/pgoyal/spark_files/json_ex.json')
data = inputJson.map(lambda x : json.loads(x))
data.collect()

[{'lovesPandas': 'Yes', 'name': 'Palash1'},
 {'lovesPandas': 'Yes', 'name': 'Palash2'},
 {'lovesPandas': 'Yes', 'name': 'Palash3'}]

In [9]:
# mapPartitions() : to reuse the JSON parser, in case of multiple JSON files 

In [15]:
# Use the above converted structured RDD to write out using Spark's text file API
(data.filter(lambda x : x['lovesPandas']).map(lambda x : json.dumps(x))).saveAsTextFile("hdfs:///user/pgoyal/spark_files/json_out.txt")

# This command will create a directory with name 'json_out.txt', and it will have 2 part files with the data partitioned

In [16]:
# CSV/TSV :
# For csv/tsv files, the header values are made as names of each field, as unlike json here the names are not attached
#     to each of the entry

In [39]:
import csv
from io import StringIO
import numpy as np
import io

def loadRecord(line):
    # This loads the data and then parses line by line
#    input = np.genfromtxt(io.BytesIO(line.encode()), delimiter=',', autostrip=True, dtype=[('Name','S5'), ('FavBook','S5')])
    input = StringIO.StringIO(line)
    reader = csv.DictReader(input, fieldnames= ["Name","FavBook"])
    return next(reader)

inputCsv = sc.textFile("hdfs:///user/pgoyal/spark_files/csv_test.csv").map(loadRecord)
inputCsv.collect()

# This is giving error (issue with reading CSV)

In [43]:
# Working example for reading CSV
# running map() to split all lines and then using 2 columns data for output
sc.textFile("hdfs:///user/pgoyal/spark_files/csv_test.csv").map(lambda 
                                                                line : line.split(",")).filter(lambda 
                                                                                               line : len(line)>1).map(lambda 
                                                                                                                       line : (line[0],line[1])).collect()


[('Name', 'FavBook'),
 ('Palash1', 'Pandas'),
 ('palash2', 'Numpy'),
 ('Palash3', 'MLLIB')]

In [36]:
# This loads all the records for a given file and then run the parser
# issue with StringIO library

import csv
import StringIO

def loadRecords(fileNameContents):
    input = StringIO.StringIO(fileNameContents[1])
    reader = csv.DictReader(input, fieldnames= ["Name","FavBook"])
    return reader

inputCsv = sc.wholeTextFiles("hdfs:///user/pgoyal/spark_files/csv_test.csv").flatMap(loadRecords)
inputCsv.collect()

# This is giving error (issue with reading CSV)

In [ ]:
# Saving CSV 
import csv
import StringIO

def saveRecords(records):
    output = StringIO.StringIO()
    writer = csv.DictReader(input, fieldnames= ["Name","FavBook"])
    for record in records:
        writer.writerow(record)
    return [output.getValue()]

#inputCsv = sc.wholeTextFiles("hdfs:///user/pgoyal/spark_files/csv_test.csv").flatMap(loadRecords)
inputCsv.mapPartitions(saveRecords).saveAsTextFile("hdfs:///user/pgoyal/spark_file/<name>")

# This is not working : StringIO error

In [ ]:
# Sequence Files : hadoop file format with key/value pairs
# Each spark datatype has corresponding Hadoop format : IntWritable, FloatWritable, etc
# Make Custom Writable format by changing 'readFields' and 'write'

In [9]:
data = sc.sequenceFile( "hdfs:///user/pgoyal/spark_files/json_out.txt/part-00000" , "org.apache.hadoop.io.Text")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.sequenceFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 7, laphdpwrk06.asia.bigdata.intraxa): java.io.IOException: hdfs://bigplay-asia-nameservice/user/pgoyal/spark_files/json_out.txt/part-00000 not a SequenceFile
	at org.apache.hadoop.io.SequenceFile$Reader.init(SequenceFile.java:1925)
	at org.apache.hadoop.io.SequenceFile$Reader.initialize(SequenceFile.java:1878)
	at org.apache.hadoop.io.SequenceFile$Reader.<init>(SequenceFile.java:1827)
	at org.apache.hadoop.io.SequenceFile$Reader.<init>(SequenceFile.java:1841)
	at org.apache.hadoop.mapred.SequenceFileRecordReader.<init>(SequenceFileRecordReader.java:49)
	at org.apache.hadoop.mapred.SequenceFileInputFormat.getRecordReader(SequenceFileInputFormat.java:64)
	at org.apache.spark.rdd.HadoopRDD$$anon$1.<init>(HadoopRDD.scala:237)
	at org.a

In [ ]:
# Using Spark SQL's HiveContext to save data from JSON file as table and then querying it
# Spark SQL - for structured and semi-structured data (ie. consistent schema across data records)

In [ ]:
# spark SQL - returns RDD of Row objects, one per record, which allow access via column number
# Each Row has get() method to return the type to cast : getFloat(), getInt(), etc.
# to access elements : row.column_name or row[column_name]

In [3]:
# Creating HiveContext object 

from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)
#rows = hiveCtx.sql("select * from axa_life_japan.adj_hist limit 10")
print rows.first().name()

In [4]:
jsonEntry = hiveCtx.jsonFile("hdfs:///user/pgoyal/spark_files/json_ex.json")
# schema has to be same throughout

/opt/cloudera/parcels/CDH/lib/spark/python/pyspark/sql/context.py:478: UserWarning: jsonFile is deprecated. Use read.json() instead.
  warnings.warn("jsonFile is deprecated. Use read.json() instead.")


In [5]:
jsonEntry.registerTempTable("json_test")
results = hiveCtx.sql("select * from json_test")

In [6]:
results.collect()

[Row(lovesPandas='Yes', name='Palash1'),
 Row(lovesPandas='Yes', name='Palash2'),
 Row(lovesPandas='Yes', name='Palash3')]